In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

# Load processor and model
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

# Load and crop the image
image_path = "/content/invoice1.jpg"
image = Image.open(image_path).convert("RGB")

# Example crop: (left, top, right, bottom)
# Adjust these values based on your image
cropped_image = image.crop((50, 50, 300, 150))

# Preprocess the cropped image
pixel_values = processor(images=cropped_image, return_tensors="pt").pixel_values

# Generate output
with torch.no_grad():
    generated_ids = model.generate(pixel_values)

# Decode predicted text
predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Predicted Text:", predicted_text)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Predicted Text: APPO 1


In [2]:
import cv2
import numpy as np
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch

# Load model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# Load image and convert to grayscale
image_path = "/content/invoice1.jpg"
original = cv2.imread(image_path)
gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

# Threshold to get binary image
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Dilate to connect text components in a line
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 5))
dilated = cv2.dilate(thresh, kernel, iterations=2)

# Find contours (each contour corresponds to a line of text)
contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours top to bottom
bounding_boxes = sorted([cv2.boundingRect(c) for c in contours], key=lambda b: b[1])

# Run OCR on each cropped line
for i, (x, y, w, h) in enumerate(bounding_boxes):
    line_img = original[y:y+h, x:x+w]
    pil_img = Image.fromarray(cv2.cvtColor(line_img, cv2.COLOR_BGR2RGB))

    # Prepare input for TrOCR
    pixel_values = processor(images=pil_img, return_tensors="pt").pixel_values

    # Predict text
    with torch.no_grad():
        generated_ids = model.generate(pixel_values)
    predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(f"Line {i+1}: {predicted_text}")


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

Line 1: 2
Line 2: 0
Line 3: 7.
Line 4: tt
Line 5: 4
Line 6: 1
Line 7: 1
Line 8: 0
Line 9: t
Line 10: d
Line 11: e
Line 12: l
Line 13: l
Line 14: 1
Line 15: b.
Line 16: r
Line 17: 0
Line 18: c
Line 19: i
Line 20: c
Line 21: l
Line 22: j.
Line 23: 11
Line 24: ul
Line 25: fl
Line 26: mm
Line 27: ll
Line 28: a
Line 29: c
Line 30: r.
Line 31: ji
Line 32: il
Line 33: c
Line 34: 2
Line 35: 0
Line 36: t
Line 37: y
Line 38: a
Line 39: p.
Line 40: 0


KeyboardInterrupt: 

In [6]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import json

# Load the processor and model
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model.eval()

# Load and preprocess the image
image = Image.open("/content/invoice1.jpg").convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values

# Define the task prompt
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

# Generate the output
with torch.no_grad():
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=512,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id
    )

# Decode and process the output
sequence = processor.batch_decode(outputs, skip_special_tokens=True)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "").strip()

# Attempt to parse the output as JSON
try:
    data = json.loads(sequence)
except json.JSONDecodeError:
    data = {"raw_output": sequence}

# Display the extracted data
print("\n📄 Extracted Invoice Data:")
for key, value in data.items():
    print(f"{key}: {value}")


preprocessor_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.02M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/806M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    1280,
    960
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cro

model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(



📄 Extracted Invoice Data:
raw_output: <s_menu><s_nm> BILLED TO: Really Great Company</s_nm><s_num> PAY TO: Avcry Davis</s_nm><s_num> 123 Anywhere SL., Any City</s_price><sep/><s_nm> Bank Really Great Bank</s_nm><s_num> 123 A56搜尋</s_num><s_price> #1024</s_price><sep/><s_nm> Account Name John Smith BSB</s_nm><s_num> ABCOURL Account Number</s_num><s_price> #1024</s_price><sep/><s_nm> RATE HOURS DESCRIPTION</s_nm><s_num> HOURS</s_num><s_price> AMOUNT</s_num><s_price> $200.00</s_price><sep/><s_nm> Content Plan</s_nm><s_num> $50/hr</s_num><s_price> $200.00</s_price><sep/><s_nm> Copy Writing</s_nm><s_num> $50/hr</s_num><s_price> $100.00</s_price><sep/><s_nm> Website Design</s_nm><s_num> $50/hr</s_num><s_price> $250.00</s_price><sep/><s_nm> Website Development</s_nm><s_num> $100/hr</s_num><s_price> $500.00</s_price><sep/><s_nm> SEO</s_nm><s_num> $50/hr</s_num><s_price> $200.00</s_price></s_menu><s_sub_total><s_subtotal_price> $1,250.00</s_subtotal_price><s_discount_price> $375.00</s_discount_

In [7]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import json

# Load model and processor
model_name = "naver-clova-ix/donut-base-finetuned-cord-v2"
processor = DonutProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)
model.eval()

# Load invoice image
image = Image.open("/content/invoice1.jpg").convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values

# Use prompt for CORD-style documents
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

# Generate output
with torch.no_grad():
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=512,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id
    )

# Decode and clean output
sequence = processor.batch_decode(outputs, skip_special_tokens=True)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "").strip()

# Try to parse output as JSON
try:
    data = json.loads(sequence)
except json.JSONDecodeError:
    data = {"raw_output": sequence}

# Extract desired fields (adjust keys depending on the actual JSON)
invoice_number = data.get("invoice_number") or data.get("Invoice No") or "Not Found"
account_number = data.get("account_number") or data.get("Account No") or "Not Found"
invoice_amount = data.get("total_amount") or data.get("Total") or "Not Found"

# Display extracted values
print("\n🧾 Key Invoice Fields:")
print(f"Invoice Number: {invoice_number}")
print(f"Account Number: {account_number}")
print(f"Invoice Amount: {invoice_amount}")


Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    1280,
    960
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cro


🧾 Key Invoice Fields:
Invoice Number: Not Found
Account Number: Not Found
Invoice Amount: Not Found


In [11]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import json

# Load model and processor
model_name = "naver-clova-ix/donut-base-finetuned-cord-v2"
processor = DonutProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)
model.eval()

# Load invoice image
image = Image.open("/content/invoice1.jpg").convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values

# Use prompt for CORD-style documents
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

# Generate output
with torch.no_grad():
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=512,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id
    )

# Decode and clean output
sequence = processor.batch_decode(outputs, skip_special_tokens=True)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "").strip()

# Try to parse output as JSON
try:
    data = json.loads(sequence)
    # If successful, print the parsed data
    print("\n🧾 Parsed JSON Output:")
    print(json.dumps(data, indent=4))  # Pretty-print the parsed JSON
except json.JSONDecodeError:
    # If JSON parsing fails, print the raw output
    print("\n🚨 Failed to parse as JSON, raw output:")
    print(sequence)


Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    1280,
    960
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cro


🚨 Failed to parse as JSON, raw output:
<s_menu><s_nm> BILLED TO: Really Great Company</s_nm><s_num> PAY TO: Avcry Davis</s_nm><s_num> 123 Anywhere SL., Any City</s_price><sep/><s_nm> Bank Really Great Bank</s_nm><s_num> 123 A56搜尋</s_num><s_price> #1024</s_price><sep/><s_nm> Account Name John Smith BSB</s_nm><s_num> ABCOURL Account Number</s_num><s_price> #1024</s_price><sep/><s_nm> RATE HOURS DESCRIPTION</s_nm><s_num> HOURS</s_num><s_price> AMOUNT</s_num><s_price> $200.00</s_price><sep/><s_nm> Content Plan</s_nm><s_num> $50/hr</s_num><s_price> $200.00</s_price><sep/><s_nm> Copy Writing</s_nm><s_num> $50/hr</s_num><s_price> $100.00</s_price><sep/><s_nm> Website Design</s_nm><s_num> $50/hr</s_num><s_price> $250.00</s_price><sep/><s_nm> Website Development</s_nm><s_num> $100/hr</s_num><s_price> $500.00</s_price><sep/><s_nm> SEO</s_nm><s_num> $50/hr</s_num><s_price> $200.00</s_price></s_menu><s_sub_total><s_subtotal_price> $1,250.00</s_subtotal_price><s_discount_price> $375.00</s_discount

In [12]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import json

# Model Names (Replace with correct model names for your use case)
models = {
    "TrOCR": "microsoft/trocr-base-handwritten",
    "Qwen2.5-VL": "Qwen2.5-VL-model-path",
    "Llama3.2-Vision": "Llama-3.2-Vision-model-path"
}

# Load TrOCR Processor and Model
def load_trocr():
    model_name = models["TrOCR"]
    processor = DonutProcessor.from_pretrained(model_name)
    model = VisionEncoderDecoderModel.from_pretrained(model_name)
    model.eval()
    return processor, model

# Load Qwen2.5-VL (Example placeholder, replace with actual load method)
def load_qwen2_5_vl():
    model_name = models["Qwen2.5-VL"]
    # Replace this with correct loading code for Qwen2.5-VL
    pass

# Load Llama 3.2 Vision (Example placeholder, replace with actual load method)
def load_llama_3_2_vision():
    model_name = models["Llama3.2-Vision"]
    # Replace this with correct loading code for Llama3.2-Vision
    pass

# Function to extract fields using model
def extract_fields_from_image(image_path, processor, model):
    # Load image
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values

    # Generate output
    task_prompt = "<s_cord-v2>"  # or other prompts as needed for each model
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    with torch.no_grad():
        outputs = model.generate(
            pixel_values,
            decoder_input_ids=decoder_input_ids,
            max_length=512,
            early_stopping=True,
            pad_token_id=processor.tokenizer.pad_token_id,
            eos_token_id=processor.tokenizer.eos_token_id
        )

    # Decode and clean output
    sequence = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "").strip()

    # Attempt to parse JSON if structured data
    try:
        data = json.loads(sequence)
        return data
    except json.JSONDecodeError:
        return {"raw_output": sequence}

# Main function to compare models
def compare_models(image_path):
    # Load models
    trocr_processor, trocr_model = load_trocr()
    qwen2_5_vl_processor, qwen2_5_vl_model = load_qwen2_5_vl()
    llama_3_2_vision_processor, llama_3_2_vision_model = load_llama_3_2_vision()

    # Extract data from TrOCR model
    trocr_data = extract_fields_from_image(image_path, trocr_processor, trocr_model)
    print("\nTrOCR Output:")
    print(json.dumps(trocr_data, indent=4))

    # Extract data from Qwen2.5-VL model (placeholder)
    # qwen2_5_vl_data = extract_fields_from_image(image_path, qwen2_5_vl_processor, qwen2_5_vl_model)
    # print("\nQwen2.5-VL Output:")
    # print(json.dumps(qwen2_5_vl_data, indent=4))

    # Extract data from Llama 3.2 Vision model (placeholder)
    # llama_3_2_vision_data = extract_fields_from_image(image_path, llama_3_2_vision_processor, llama_3_2_vision_model)
    # print("\nLlama 3.2 Vision Output:")
    # print(json.dumps(llama_3_2_vision_data, indent=4))

    # You can now compare extracted data from each model, for instance:
    trocr_invoice_date = trocr_data.get("invoice_date", "Not Found")
    trocr_invoice_amount = trocr_data.get("invoice_amount", "Not Found")

    # Similar comparisons for Qwen2.5-VL and Llama 3.2 Vision (if uncommented)

    # Final comparison:
    print("\n✅ Final Extracted Fields Comparison:")
    print(f"Invoice Date (TrOCR): {trocr_invoice_date}")
    print(f"Invoice Amount (TrOCR): {trocr_invoice_amount}")

# Run comparison
compare_models("/content/invoice1.jpg")


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

TypeError: cannot unpack non-iterable NoneType object

In [1]:
# Install the required packages if not already installed
# pip install transformers torch torchvision pillow

from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

# Load the image
image_path = "/content/inv.jpeg"  # Replace with your actual path
image = Image.open(image_path).convert("RGB")

# Load the TrOCR processor and model (base model for printed text)
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
model.eval()

# Preprocess the image and generate output
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Generate text using the model
with torch.no_grad():
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Print the full raw extracted text
print("📄 Extracted Text:\n")
print(generated_text)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📄 Extracted Text:

CHANGE


In [2]:
import re

# Try extracting a line that contains "Total Amount Paid"
matches = re.findall(r"Total Amount Paid\s*₹?\s*[\d,]+", generated_text, re.IGNORECASE)
if matches:
    print("\n💰 Total Amount Paid Found:")
    print(matches[0])
else:
    print("\n⚠️ 'Total Amount Paid' not found explicitly in the OCR output.")



⚠️ 'Total Amount Paid' not found explicitly in the OCR output.


In [4]:
# Install dependencies if not done already
# pip install transformers torch torchvision pillow opencv-python

from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import cv2
import numpy as np

# Load TrOCR model
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
model.eval()

# Load and preprocess image
image_path = "/content/inv.jpeg"  # <-- Replace this
image_pil = Image.open(image_path).convert("RGB")
image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold and invert
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours (likely text lines)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by y position (top to bottom)
bounding_boxes = [cv2.boundingRect(c) for c in contours]
sorted_boxes = sorted(bounding_boxes, key=lambda b: b[1])

print("🔍 Extracted Lines:\n")
extracted_lines = []

for i, (x, y, w, h) in enumerate(sorted_boxes):
    if h < 15:  # Skip very small boxes (likely noise)
        continue

    # Crop line
    cropped_line = image_pil.crop((x, y, x + w, y + h))

    # Apply TrOCR
    pixel_values = processor(images=cropped_line, return_tensors="pt").pixel_values
    with torch.no_grad():
        generated_ids = model.generate(pixel_values)
        text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        extracted_lines.append(text)
        print(f"{i+1}. {text}")

# 🔎 Optional: Extract the line containing "Total Amount Paid"
print("\n💰 Trying to find 'Total Amount Paid':")
for line in extracted_lines:
    if "total amount paid" in line.lower():
        print(f"✅ Found)


SyntaxError: unterminated string literal (detected at line 53) (<ipython-input-4-98b53223a8c4>, line 53)

In [5]:
# Install dependencies if not done already
# pip install transformers torch torchvision pillow opencv-python

from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import cv2
import numpy as np

# Load TrOCR model
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
model.eval()

# Load and preprocess image
image_path = "/content/inv.jpeg"  # <-- Replace this
image_pil = Image.open(image_path).convert("RGB")
image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold and invert
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours (likely text lines)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by y position (top to bottom)
bounding_boxes = [cv2.boundingRect(c) for c in contours]
sorted_boxes = sorted(bounding_boxes, key=lambda b: b[1])

print("🔍 Extracted Lines:\n")
extracted_lines = []

for i, (x, y, w, h) in enumerate(sorted_boxes):
    if h < 15:  # Skip very small boxes (likely noise)
        continue

    # Crop line
    cropped_line = image_pil.crop((x, y, x + w, y + h))

    # Apply TrOCR
    pixel_values = processor(images=cropped_line, return_tensors="pt").pixel_values
    with torch.no_grad():
        generated_ids = model.generate(pixel_values)
        text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        extracted_lines.append(text)
        print(f"{i+1}. {text}")

# 🔎 Optional: Extract the line containing "Total Amount Paid"
print("\n💰 Trying to find 'Total Amount Paid':")
for line in extracted_lines:
    if "total amount paid" in line.lower():
        print(f"✅ Found: {line}")
        break
else:
    print("❌ 'Total Amount Paid' not found in lines.")


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

🔍 Extracted Lines:

1. <
2. <
4. 9
5. 3
6. 6
7. 8
8. 2
9. 6
10. 1
11. 3
12. 7
13. 4
14. 2
15. D
16. :
17. K
18. B
19. 9
20. 0
21. :
22. O
23. O
26. 1
27. D
28. P
29. IT
30. IT


KeyboardInterrupt: 

In [6]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import cv2
import numpy as np

# Load TrOCR model
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
model.eval()

# Load image
image_path = "/content/inv.jpeg"
image_pil = Image.open(image_path).convert("RGB")
image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold for text region detection
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Dilate to combine nearby text into lines
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (image.shape[1]//20, 5))
dilated = cv2.dilate(thresh, kernel, iterations=1)

# Find contours
contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours top-to-bottom
bounding_boxes = sorted([cv2.boundingRect(c) for c in contours], key=lambda b: b[1])

# Run OCR on each line
print("📝 Extracted Lines:\n")
extracted_lines = []

for i, (x, y, w, h) in enumerate(bounding_boxes):
    if h < 15 or w < 50:
        continue

    # Add padding to each line box
    pad = 10
    x1 = max(x - pad, 0)
    y1 = max(y - pad, 0)
    x2 = min(x + w + pad, image.shape[1])
    y2 = min(y + h + pad, image.shape[0])

    # Crop and convert to PIL
    line_img = image[y1:y2, x1:x2]
    line_pil = Image.fromarray(cv2.cvtColor(line_img, cv2.COLOR_BGR2RGB))

    # OCR using TrOCR
    pixel_values = processor(images=line_pil, return_tensors="pt").pixel_values
    with torch.no_grad():
        generated_ids = model.generate(pixel_values)
    line_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    extracted_lines.append(line_text)
    print(f"{i+1}. {line_text}")

# Optional: Find line with 'Total Amount Paid'
print("\n🔎 Line containing 'Total Amount Paid':")
for line in extracted_lines:
    if "total amount paid" in line.lower():
        print(f"✅ {line}")
        break
else:
    print("❌ Not found.")


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

📝 Extracted Lines:

1. <
2. <
3. BOOKING ID - 24731628639
4. PAYMENT DETAILS
5. BASE FARE
6. R 4,910
7. R1,439
8. TAXES AND FEES
9. TOTAL AIRFARE
10. $6,349
11. R 349
12. CONVENIENCE FEE
13. INSTANT DISCOUNT
14. # 400
15. TOTAL AMOUNT PAID
16. # 6,298
17. R 6,298
18. UPL
19. TRANSACTION ID #502943232268

🔎 Line containing 'Total Amount Paid':
✅ TOTAL AMOUNT PAID


In [9]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import json
import re

# Load model and processor
model_name = "naver-clova-ix/donut-base-finetuned-cord-v2"
processor = DonutProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)
model.eval()

# Load invoice image
image = Image.open("/content/inv.jpeg").convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values

# Use prompt for CORD-style documents
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

# Generate output
with torch.no_grad():
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=768,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id
    )

# Decode output
sequence = processor.batch_decode(outputs, skip_special_tokens=True)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "").strip()

# Print raw JSON-like output
print("\n📄 Raw Donut Output:")
print(sequence)

# Try to parse output as JSON
try:
    data = json.loads(sequence)
except json.JSONDecodeError:
    print("\n❌ Couldn't parse output as JSON. Wrapping in fallback JSON.")
    data = {"raw_output": sequence}

# Optional: extract values from raw text if needed
def extract_from_raw(text, field_name):
    pattern = re.compile(rf"{field_name}[^0-9]*([\d,]+)", re.IGNORECASE)
    match = pattern.search(text)
    return match.group(1).replace(",", "") if match else "Not Found"

# Extract relevant fields from JSON or raw text
raw_text = json.dumps(data)
total_paid = extract_from_raw(raw_text, "total amount paid")
transaction_id = extract_from_raw(raw_text, "transaction id")
base_fare = extract_from_raw(raw_text, "base fare")
taxes_fees = extract_from_raw(raw_text, "taxes and fees")

# Final output
print("\n🧾 Extracted Fields:")
print(f"Total Amount Paid: {total_paid}")
print(f"Transaction ID: {transaction_id}")
print(f"Base Fare: {base_fare}")
print(f"Taxes and Fees: {taxes_fees}")


Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    1280,
    960
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cro


📄 Raw Donut Output:
<s_menu><s_nm> Booking ID-24731628639</s_nm><s_unitprice> 4,910</s_unitprice><s_cnt> <</s_cnt><s_price> 4,910</s_price><s_sub><s_nm> Taxes and Fees</s_nm><s_price> 1,439</s_price></s_sub></s_menu><s_sub_total><s_subtotal_price> 6,349</s_subtotal_price><s_discount_price> 349</s_discount_price></s_sub_total><s_total><s_total_price> 6,298</s_total_price><s_total_etc> 6,298</s_total_etc><s_creditcardprice> 6,298</s_creditcardprice></s_total>

❌ Couldn't parse output as JSON. Wrapping in fallback JSON.

🧾 Extracted Fields:
Total Amount Paid: Not Found
Transaction ID: Not Found
Base Fare: Not Found
Taxes and Fees: 1439


In [10]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import cv2
import numpy as np

# Load Donut model and processor
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model.eval()

# Load image
image_path = "/content/inv.jpeg"
image_pil = Image.open(image_path).convert("RGB")
image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold for text region detection
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Dilate to combine nearby text into lines
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (image.shape[1] // 20, 5))
dilated = cv2.dilate(thresh, kernel, iterations=1)

# Find contours
contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours top-to-bottom
bounding_boxes = sorted([cv2.boundingRect(c) for c in contours], key=lambda b: b[1])

# Run OCR on each line
print("📝 Extracted Lines using Donut:\n")
extracted_lines = []

for i, (x, y, w, h) in enumerate(bounding_boxes):
    if h < 15 or w < 50:
        continue

    # Add padding to each line box
    pad = 10
    x1 = max(x - pad, 0)
    y1 = max(y - pad, 0)
    x2 = min(x + w + pad, image.shape[1])
    y2 = min(y + h + pad, image.shape[0])

    # Crop and convert to PIL
    line_img = image[y1:y2, x1:x2]
    line_pil = Image.fromarray(cv2.cvtColor(line_img, cv2.COLOR_BGR2RGB))

    # Preprocess image
    pixel_values = processor(images=line_pil, return_tensors="pt").pixel_values

    # Donut requires a task prompt — use a generic one
    decoder_input_ids = processor.tokenizer("<s_docvqa>", add_special_tokens=False, return_tensors="pt")["input_ids"]

    with torch.no_grad():
        outputs = model.generate(
            pixel_values,
            decoder_input_ids=decoder_input_ids,
            max_length=512,
            early_stopping=True,
            pad_token_id=processor.tokenizer.pad_token_id,
        )

    line_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    extracted_lines.append(line_text)
    print(f"{i+1}. {line_text}")

# Optional: Find line with 'Total Amount Paid'
print("\n🔎 Line containing 'Total Amount Paid':")
for line in extracted_lines:
    if "total amount paid" in line.lower():
        print(f"✅ {line}")
        break
else:
    print("❌ Not found.")


preprocessor_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/803M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    2560,
    1920
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cr

📝 Extracted Lines using Donut:



model.safetensors:   0%|          | 0.00/803M [00:00<?, ?B/s]

1. ®
2. .
3. booking id - 24731628639
4. tayment details
5. base
6. ral. 910
7. , 4 3
8. taxes and fees
9. total airfare
10. rka
11. ral
12. convenience
13. instant discount
14. rt gold
15. total amount paid
16. . 29
17. rka
18. .
19. transaction id #502943232268

🔎 Line containing 'Total Amount Paid':
✅ total amount paid


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


OSError: Qwen/Qwen-VL does not appear to have a file named preprocessor_config.json. Checkout 'https://huggingface.co/Qwen/Qwen-VL/tree/main'for available files.

In [13]:
pip install transformers accelerate torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:

from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch

# Load model and processor
model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForVision2Seq.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

# Load invoice image
image_path = "/content/inv.jpeg"
image = Image.open(image_path).convert("RGB")

# Define your prompt
prompt = """
You're a helpful assistant. Extract the following fields from the invoice:
- Total Amount Paid
- Base Fare
- Taxes and Fees
- Transaction ID

Respond in this format:
{
  "total_amount_paid": "...",
  "base_fare": "...",
  "taxes_and_fees": "...",
  "transaction_id": "..."
}
"""

# Preprocess inputs, include the image token in the prompt
# The <image> token tells the model where to incorporate the image
inputs = processor(images=image, text=prompt + "<image>", return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=512)

# Decode result
result = processor.batch_decode(outputs, skip_special_tokens=True)[0]

# Show result
print("\n🧾 LLaMA 3.2 Vision Output:")
print(result)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
pip install easyocr opencv-python pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 23.3 MB/s eta 0:00:00


In [5]:
import easyocr
import cv2
import re
from PIL import Image

# Step 1: Load the image
image_path = "/content/inv.jpeg"
image = cv2.imread(image_path)

# Step 2: Initialize the OCR reader
reader = easyocr.Reader(['en'])

# Step 3: Run OCR
results = reader.readtext(image)

# Step 4: Collect and clean lines
lines = [text for (_, text, _) in results]
full_text = "\n".join(lines)
print("📝 All OCR Text:\n", full_text)

# Step 5: Extract fields using regex or keyword search
def extract_field(name, pattern):
    for line in lines:
        if name.lower() in line.lower():
            match = re.search(pattern, line)
            if match:
                return match.group(1)
    return "Not Found"

fields = {
    "total_amount_paid": extract_field("total amount paid", r"[\₹$#]?\s*([\d,]+)"),
    "base_fare": extract_field("base fare", r"[\₹$#]?\s*([\d,]+)"),
    "taxes_and_fees": extract_field("taxes and fees", r"[\₹$#]?\s*([\d,]+)"),
    "transaction_id": extract_field("transaction id", r"#?(\d{6,})")
}

# Step 6: Display extracted results
print("\n🔍 Extracted Invoice Fields:")
for key, value in fields.items():
    print(f"{key}: {value}")


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete📝 All OCR Text:
 Booking ID
24731628639
Payment Details
Base fare
? 4,910
Taxes and Fees
71,439
Total Airfare
?
6,349
Convenience Fee
< 349
Instant Discount
400
Total Amount Paid
8 6,298
UPI
? 6,298
Transaction ID #502943232268

🔍 Extracted Invoice Fields:
total_amount_paid: Not Found
base_fare: Not Found
taxes_and_fees: Not Found
transaction_id: 502943232268
